# Identification of NI using method 2 (LOS>15 days)

Notes: - Includes data from 1st January 2020 - 31st March 2021

In [55]:
# Importing Libraries
import import_ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from importlib import reload

# Importing Notebooks
import NI_M2
reload(NI_M2)


importing Jupyter notebook from NI_M2.ipynb


<module 'NI_M2' from 'NI_M2.ipynb'>

## Importing and reading datasets

In [2]:

SaveCSV = False
ReadExcel = True

# Optional: Reading Excel file and saving csv file
if SaveCSV:
    read_file = pd.read_excel(r'O:\DiRAC\FlavienHardy\2_NosocomialInfections\Data\0_Raw\MarchData\All episodes 07-06-21.xlsx', sheet_name='Sheet1')
    read_file.to_csv(r'O:\DiRAC\FlavienHardy\2_NosocomialInfections\Data\1_Processed\All episodes 07-06-21.csv', index=None, header=True)

if ReadExcel:
    # Importing dataset
    df_AllEpisodes = pd.read_excel(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\Data\0_Raw\MarchData\All episodes 29-06-21.xlsx')


## Pre-processing

In [56]:

# Pre-processing: homogenising date formats
df2 = df_AllEpisodes
df2 = NI_M2.PreProcessing(df2)

ColumnList = ['HESID', 'diagnosis_group', 'Admission_date', 'Discharge_date', 'EPIstart', 'EPIend', 'Diff_EPIstart_Admission']
print('Size of dataset:', df2.shape)
df2[ColumnList].head(2)


Size of dataset: (486304, 18)


,HESID,diagnosis_group,Admission_date,Discharge_date,EPIstart,EPIend,Diff_EPIstart_Admission
23777,000092DDD60956DDAD79301E546F2D9B,U072~J189~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~...,2020-03-24,2020-04-03,2020-03-25,2020-03-30,1
26405,0000FFD16F52D58C19C8EE80D07B9435,U071~J128~B972~J9690~B972~F059~R268~R464~I489~...,2020-03-26,2020-07-27,2020-03-26,2020-04-28,0


## Implementing methods 2

In [79]:

ColumnNames = []
Ndays = [15, 8]
for ndays in Ndays:
    df2 = NI_M2.NI_M2_Identification(df2, ndays)
    ColumnNames.append('NC_Method2_'+str(ndays))

# Extract patient / Method 2 classification
ColumnList = ['P_Spell_ID','HESID', 'diagnosis_group', 'Admission_date', 'Discharge_date', 'EPIstart', 'Diff_EPIstart_Admission']+ColumnNames
df_M2 = df2[ColumnList]

print('Size of dataset:', df_M2.shape)
df_M2.head(5)

if len( df_M2['P_Spell_ID'] ) == len(set(df_M2['P_Spell_ID'])):
    print('Contains UNIQUE spells')
else:
    print('Contains DUPLICATE spells')
    
if len( df_M2['HESID'] ) == len(set(df_M2['HESID'])):
    print('Contains UNIQUE patients.')
else:
    print('Contains DUPLICATE patients; number of UNIQUE patients =', len(set(df_M2['HESID'])))

df_M2.head(2)


Size of dataset: (486304, 9)
Contains UNIQUE spells
Contains DUPLICATE patients; number of UNIQUE patients = 382329


,P_Spell_ID,HESID,diagnosis_group,Admission_date,Discharge_date,EPIstart,Diff_EPIstart_Admission,NC_Method2_15,NC_Method2_8
23777,1656,000092DDD60956DDAD79301E546F2D9B,U072~J189~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~-1~...,2020-03-24,2020-04-03,2020-03-25,1,0,0
26405,2864,0000FFD16F52D58C19C8EE80D07B9435,U071~J128~B972~J9690~B972~F059~R268~R464~I489~...,2020-03-26,2020-07-27,2020-03-26,0,0,0


In [80]:

# Print characteristics for two methods 2
Ndays = [15, 8]
for ndays in Ndays:
    
    NameColumn = 'NC_Method2_'+str(ndays)
    N_NI_M2 = df_M2[NameColumn].sum()
    N_Infections = df_M2[NameColumn].count()
    NI_Prop_NI = 100*N_NI_M2/N_Infections
    
    print('-----------------------')
    print('Method 2 using ' + str(ndays) + ' days cut-off:')
    print('Total number of infections in dataset: ', N_Infections)
    print('Number of identified NI using M2: ', N_NI_M2)
    print('Proportion of identified NI using M2: ', NI_Prop_NI, '%')


-----------------------
Method 2 using 15 days cut-off:
Total number of infections in dataset:  486304
Number of identified NI using M2:  9469
Proportion of identified NI using M2:  1.9471359478844508 %
-----------------------
Method 2 using 8 days cut-off:
Total number of infections in dataset:  486304
Number of identified NI using M2:  19498
Proportion of identified NI using M2:  4.009426202539975 %


In [81]:

# Save M2 dataframe
Save = True
path = r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\src\0.processing\data\NI_M2.csv'
if Save:
    df_M2.to_csv(r'O:\GIRFT DiRAC\FlavienHardy\2_NosocomialInfections\src\0.processing\data\NI_M2.csv')
